In [19]:
import gradio as gr
from audiolm.model import AudioLM
import os
from pathlib import Path
import torch
import numpy as np

audiolm_instance = AudioLM.from_pretrained(Path(os.getcwd() + "\\..\\data\\"))
def generate(audio):
    sr, x = audio
    return audiolm_instance.generate(x)
# Create the Gradio interface
audiolm_app = gr.Interface(
    fn=lambda sr,audio: print(sr,audio),#audiolm_instance.generate(audio),
    inputs=gr.Audio(sources=["microphone","upload"]),
    outputs=gr.Audio(),
    title="AudioLM",
    description="Upload an audio file to reverse its content."
)


audiolm_app.launch()

c:\Users\josed\Documents\AudioLM\.venv\lib\site-packages\torch\nn\utils\weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
c:\Users\josed\Documents\AudioLM\.venv\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MiniBatchKMeans from version 0.24.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\josed\Documents\AudioLM\.venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_d

Running on local URL:  http://127.0.0.1:7872

To create a public link, set `share=True` in `launch()`.


2024-05-19 23:52:32 | INFO | httpx | HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
c:\Users\josed\Documents\AudioLM\.venv\lib\site-packages\gradio\helpers.py:947: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")


None None


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\josed\Documents\AudioLM\.venv\lib\site-packages\uvicorn\protocols\http\httptools_impl.py", line 411, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "c:\Users\josed\Documents\AudioLM\.venv\lib\site-packages\uvicorn\middleware\proxy_headers.py", line 69, in __call__
    return await self.app(scope, receive, send)
  File "c:\Users\josed\Documents\AudioLM\.venv\lib\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "c:\Users\josed\Documents\AudioLM\.venv\lib\site-packages\starlette\applications.py", line 123, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\Users\josed\Documents\AudioLM\.venv\lib\site-packages\starlette\middleware\errors.py", line 186, in __call__
    raise exc
  File "c:\Users\josed\Documents\AudioLM\.venv\lib\site-packages\starlette\middleware\errors.py",

(48000, array([ -523264,  -481024,  -435968, ..., 12931840, 12788736,        0],
      dtype=int32)) None
(48000, array([ -523264,  -481024,  -435968, ..., 12931840, 12788736,        0],
      dtype=int32)) None


In [4]:
Path(os.getcwd() + "\\..\\data\\")

WindowsPath('c:/Users/josed/Documents/AudioLM/notebooks/../data')